<a href="https://colab.research.google.com/github/MPrazzoli/corpsol/blob/main/Script_buono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
pip install QuantLib

In [165]:
import os
import math
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date

***Inserimento degli Input***

In [ ]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

***Schedule Date***

In [47]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor = Period(tenor, Months)
endofmonth = False
terminationDateConvention = business_convention
df = pd.DataFrame({'startdate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[:-1],'enddate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[1:]})

In [48]:
# Calcolo numero delle rate totali dell'ammortamento
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor) * int(maturity_label.rsplit(maturity_label[-1], 1)[0]))

***Ammortamento Francese***

In [49]:
def ammortamento_francese(amortization_rate,tenor,rate_totali,notional):
  
  t = (1 + amortization_rate / (12 / tenor)) ** rate_totali
  rata_fra = round(notional * t * (amortization_rate / (12 / tenor)) / (t - 1),2)
  Residuo_fra = notional
  Estinto_fra = 0
  tot_I_fra = 0
  plan_fra = np.array
  plan_fra = [[0,0,0,0,0, Residuo_fra]]
  rate_fra = range(0, rate_totali)
  for count in rate_fra:
    Int_quota_fra = round(Residuo_fra * (amortization_rate*(tenor/12)),2)
    Cap_quota_fra = (rata_fra - Int_quota_fra)
    tot_I_fra = (tot_I_fra + Int_quota_fra)
    Residuo_fra = (Residuo_fra - Cap_quota_fra)
    Estinto_fra = (Estinto_fra + Cap_quota_fra)
    Tot_fra = (Int_quota_fra + Cap_quota_fra)
    plan.append([Int_quota_fra, Cap_quota_fra, Tot_fra, tot_I_fra, Estinto_fra, Residuo_fra])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_fra = pd.DataFrame(plan_fra)
  df_fra.columns = index
  df1_fra = pd.concat([df,df_fra], axis=1)
  df2_fra = pd.concat([df,df_fra['Debito Residuo']], axis=1)[:-1]

  return (df2_fra)

***Ammortamento Italiano***


In [50]:
def ammortamento_italiano(amortization_rate,tenor,rate_totali, notional):

  Residuo_ita = notional
  tot_I_ita = 0
  Estinto_ita = 0
  Tot_ita = 0
  plan_ita = np.array
  plan_ita = [[0,0,0,0,0,Residuo_ita]]
  rate_ita = range(0,rate_totali)
  for count in rate_ita:
    Int_quota_ita = round(Residuo_ita*amortization_rate/(12/tenor),2)
    Cap_quota_ita = round((notional/rate_totali),2)
    tot_I_ita = (tot_I_ita + Int_quota_ita)
    Residuo_ita = (Residuo_ita - Cap_quota_ita)
    Estinto_ita = (Estinto_ita + Cap_quota_ita)
    Tot_ita = (Int_quota_ita + Cap_quota_ita)
    plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_ita = pd.DataFrame(plan_ita)
  df_ita.columns=index
  df1_ita = pd.concat([df,df_ita],axis=1)
  df2_ita = pd.concat([df,df_ita['Debito Residuo']],axis=1)[:-1]
  
  return (df2_ita)
  

***Ammortamento Bullet***


In [51]:
def ammortamento_bullet(amortization_rate,tenor,rate_totali, notional):

  Residuo_bullet = notional
  tot_I_bullet = 0
  plan_bullet = np.array
  plan_bullet = [[0,0,Residuo_bullet]]
  rate_bullet = range(0,rate_totali)
  for count in rate_bullet:
    Int_quota_bullet = round(Residuo_bullet*amortization_rate/(12/tenor),2)
    tot_I_bullet = (tot_I_bullet+Int_quota_bullet)
    plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])
  index = ['Quota Interesse','Interessi Totali','Debito Residuo']
  df_bullet = pd.DataFrame(plan_bullet)
  df_bullet.columns = index
  df1_bullet = pd.concat([df,df_bullet],axis=1)
  df2_bullet = pd.concat([df,df_bullet['Debito Residuo']],axis=1)
  df2_bullet = df2_bullet[:-1]
  
  return (df2_bullet)

***Output del Modello***


In [ ]:
if (amortization_type == 'Francese'):
  df_output = (ammortamento_francese(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Italiano'):
  df_output = (ammortamento_italiano(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Bullet'):
  df_output = (ammortamento_bullet(amortization_rate,tenor,rate_totali,notional))
print (df_output)

Npv Obiettivo: prova

In [190]:
df_output['differenza_giorni'] = (df_output['enddate'] - df_output['startdate']) / 360
df_output['prod'] = (df_output['Debito Residuo'] * df_output['differenza_giorni']) * 0.0001
BPV = df_output['prod'].sum()

*Class: Market Data*

In [64]:
class historical_fixed_rate(object):

    def __init__(self, name, reference_date = None, currency = None, underlying = None):
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
        
        self.name = name
        self.currency = currency
        self.underlying = underlying
        self.historical_fixed_rate_df = pd.DataFrame

class zero_curve_rate(object):
# chiedere a Mirko l'assegnazione di interp_rule e day_count
    def __init__(self, name, reference_date = None, currency = None, underlying = None, interp_rule = None, day_count = None):
        
        self.name = name
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
      
        self.currency = currency
        self.underlying = underlying
        self.interpolation_rule = interp_rule
        self.day_count = day_count
        self.zero_curve_rate_df = pd.DataFrame

In [69]:
path = '/content/drive/MyDrive/data.xlsx'

def market_data(path):

  historical_fixed_rates = historical_fixed_rate(name="HISTORICAL RATE")
  historical_fixed_rates.historical_fixed_rate_df = pd.read_excel(path, sheet_name='fixings')

  estimation_curve = zero_curve_rate(name='ESTIMATION')
  estimation_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='estimation')

  discount_curve = zero_curve_rate(name='DISCOUNT')
  discount_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='discount')

  market_data_dict = {'ESTIMATION': estimation_curve, 'DISCOUNT': discount_curve, 'HISTORICAL': historical_fixed_rates}

  return(market_data_dict)

In [194]:
m=market_data(path)
m['ESTIMATION'].zero_curve_rate_df
g[]

,DATE,RATE
0,2021-02-01,-0.005442
1,2021-02-05,-0.005442
2,2021-02-12,-0.005442
3,2021-02-19,-0.005442
4,2021-02-26,-0.005442
...,...,...
60,2059-01-29,0.000529
61,2060-01-29,0.000488
62,2061-01-31,0.000446
63,2071-01-29,0.000072


In [54]:
class IRS:

  def __init__(self, leg1_name = 'Receive', leg2_name = 'Pay', leg1_rate_type = 'Fixed', leg2_rate_type = 'Floating'):

    self.start_date = None
    self.leg1_flows = pd.DataFrame
    self.leg2_flows = pd.DataFrame
    self.leg1_name = leg1_name
    self.leg2_name = leg2_name
    self.leg1_rate_type = leg1_rate_type  
    self.leg2_rate_type = leg2_rate_type
    self.npv = None

In [55]:
def submit(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

  irs = IRS()

In [56]:
submit(path = market_data_path, underlying = underlying, amortization_type = amortization_type, notional = notional, amortization_rate = amortization_rate, maturity_label = maturity_label, ref_date = ref_date, tenor = tenor,
       business_convention = business_convention, date_gen_rule = date_gen_rule)

In [ ]:
leg_fixed=FixedRateLeg(schedule,Actual360(),[notional],[0.05])
for i in leg_fixed:
  print(i.date(),i.amount())